#Dependencies


In [1]:
!pip install typing-extensions
!pip install transformers
!pip install huggingface_hub
!pip install seqeval
!pip install transformers huggingface_hub
!pip install sklearn_crfsuite
!pip install transformers datasets evaluate seqeval

  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


#Imports

In [2]:
import pandas as pd
import numpy as np
import random
import torch
from transformers import BertModel, BertConfig, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import torch
from string import punctuation
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import evaluate
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import datetime

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Dataset

## download dataset from kaggle

In [3]:
!kaggle datasets download -d naseralqaydeh/named-entity-recognition-ner-corpus

Dataset URL: https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus
License(s): DbCL-1.0
 97% 4.00M/4.14M [00:01<00:00, 4.36MB/s]
100% 4.14M/4.14M [00:01<00:00, 3.27MB/s]


##unzip the folder

In [4]:
!unzip /content/named-entity-recognition-ner-corpus.zip

Archive:  /content/named-entity-recognition-ner-corpus.zip
  inflating: ner.csv                 


##read the dataset

In [5]:
data = pd.read_csv("/content/ner.csv")

In [6]:
data.head()

Sentence #                                           Sentence  \
0  Sentence: 1  Thousands of demonstrators have marched throug...   
1  Sentence: 2  Families of soldiers killed in the conflict jo...   
2  Sentence: 3  They marched from the Houses of Parliament to ...   
3  Sentence: 4  Police put the number of marchers at 10,000 wh...   
4  Sentence: 5  The protest comes on the eve of the annual con...   

                                                 POS  \
0  ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...   
1  ['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...   
2  ['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...   
3  ['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...   
4  ['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...   

                                                 Tag  
0  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...  
1  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...  
2  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...  
3  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...  
4  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  47959 non-null  object
 1   Sentence    47959 non-null  object
 2   POS         47959 non-null  object
 3   Tag         47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


# Embedding Model

##Load BERT tokenizer and model

In [8]:
bert = BertModel.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## function to get the sentence embedding

In [9]:
def get_embeddings(input_ids, attention_mask):
  with torch.no_grad():

      outputs = bert(input_ids, attention_mask=attention_mask)
  word_embeddings = outputs.last_hidden_state
  return word_embeddings

#Preprocessing & Feature extraction

##Check for nulls

In [42]:
data.isna().sum()

Sentence #    0
Sentence      0
POS           0
Tag           0
dtype: int64

In [43]:
data.isnull().sum()

Sentence #    0
Sentence      0
POS           0
Tag           0
dtype: int64

##Get unique POS and NE tags and mapping dict

In [44]:
pos_tags = []
for i in data["POS"]:
  i = i.replace("'","").replace('[','').replace(']','').replace(" ", "").split(',')[:-1]
  for j in i:
    if j not in pos_tags:
      pos_tags.append(j)
pos_tags_to_index = {pos_tags[i]:i for i in range(len(pos_tags))}
pos_tags_to_index

{'NNS': 0,
 'IN': 1,
 'VBP': 2,
 'VBN': 3,
 'NNP': 4,
 'TO': 5,
 'VB': 6,
 'DT': 7,
 'NN': 8,
 'CC': 9,
 'JJ': 10,
 'VBD': 11,
 'WP': 12,
 '``': 13,
 'CD': 14,
 '.': 15,
 'PRP': 16,
 'VBZ': 17,
 'POS': 18,
 'VBG': 19,
 'RB': 20,
 '': 21,
 'WRB': 22,
 'PRP$': 23,
 'MD': 24,
 'WDT': 25,
 'JJR': 26,
 ':': 27,
 'JJS': 28,
 'WP$': 29,
 'RP': 30,
 'PDT': 31,
 'NNPS': 32,
 'EX': 33,
 'RBS': 34,
 'LRB': 35,
 'RRB': 36,
 '$': 37,
 'RBR': 38,
 ';': 39,
 'UH': 40,
 'FW': 41}

In [45]:
tags = []
for i in data["Tag"]:
  i = i.replace("'","").replace('[','').replace(']','').replace(" ", "").split(',')[:-1]
  for j in i:
    if j not in tags:
      tags.append(j)
taps_to_index = {tags[i-1]:i for i in range(1, len(tags) + 1)}
taps_to_index

{'O': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-per': 4,
 'I-geo': 5,
 'B-org': 6,
 'I-org': 7,
 'B-tim': 8,
 'B-art': 9,
 'I-art': 10,
 'I-per': 11,
 'I-gpe': 12,
 'I-tim': 13,
 'B-nat': 14,
 'B-eve': 15,
 'I-eve': 16,
 'I-nat': 17}

#Build feature Dict

##Features extraction for CRF

In [46]:
def sentence2feat_CRF(sentence, pos):
  sentence = sentence.split()
  feat = []
  for i in range(len(sentence)):
    word_feat = {}
    word_feat['word'] = sentence[i]
    word_feat['pos'] = pos[i]
    word_feat['is_capitalizes'] = 1 if sentence[i][0].isupper() else 0
    word_feat['is_all_caps'] = 1 if sentence[i].isupper() else 0
    word_feat['lower_case'] = sentence[i].lower()
    word_feat['shape'] = ''.join(["X" if j.isupper() else 'x' for j in sentence[i]])

    if len(sentence) == 1:
      word_feat['prev_word'] = '<S>'
      word_feat['prev_pos'] = '<S>'
      word_feat['next_word'] = '<E>'
      word_feat['next_pos'] = '<E>'
      feat.append(word_feat)
      continue

    if i == 0:
      word_feat['prev_word'] = '<S>'
      word_feat['prev_pos'] = '<S>'
      word_feat['next_word'] = sentence[i+1]
      word_feat['next_pos'] = pos[i+1]

    elif i == len(sentence)-1:
      word_feat['prev_word'] = sentence[i-1]
      word_feat['prev_pos'] = pos[i-1]

    elif i == 1:
      word_feat['prev_word'] = sentence[i-1]
      word_feat['prev_pos'] = pos[i-1]
      word_feat['next_word'] = sentence[i+1]
      word_feat['next_pos'] = pos[i+1]

    else:
      word_feat['prev_word'] = sentence[i-1]
      word_feat['next_word'] = sentence[i+1]
      word_feat['prev_pos'] = pos[i-1]
      word_feat['next_pos'] = pos[i+1]

    feat.append(word_feat)

  return feat


#CRF Alogorithm

##Create dataset

In [47]:
dataset = []
labels = []
for sentence, pos, label in zip(data["Sentence"], data["POS"], data["Tag"]):
  pos = pos.replace("'","").replace('[','').replace(']','').split(',')
  if len(sentence.split()) != len(pos):
    continue
  dataset.append(sentence2feat_CRF(sentence, pos))
  labels.append(label.replace("'","").replace('[','').replace(']','').split(','))

##Split Dataset

In [48]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42, shuffle=True)

##Train CRF and make prediction

In [49]:
crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)
crf.fit(X_train, y_train)
y_pred = crf.predict(X_test)

##Evaluate Predictions

In [50]:
for true_seq, pred_seq in zip(y_test, y_pred):
    assert len(true_seq) == len(pred_seq), "Mismatch in sequence lengths"

print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1-Score: {f1_score(y_test, y_pred)}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Precision: 0.8674814073848562
Recall: 0.8388073510240128
F1-Score: 0.8529034465064569

Classification Report:
              precision    recall  f1-score   support

           O       0.91      0.89      0.90     13045
         art       0.00      0.00      0.00        30
         eve       0.00      0.00      0.00        29
         geo       0.78      0.90      0.84      3571
         gpe       0.92      0.89      0.91      1662
         nat       0.00      0.00      0.00        23
         org       0.76      0.64      0.69      1921
         per       0.74      0.68      0.71      1652
         tim       0.92      0.71      0.80      1846

   micro avg       0.87      0.84      0.85     23779
   macro avg       0.56      0.52      0.54     23779
weighted avg       0.87      0.84      0.85     23779



#Create Dataset of Feature map and label

feature map is the concatination of sentence embedding with the one hot encoded POS tags

the dataset contains some instances with missing POS tags

solution: pre-traind POS tagger

##Get unique tags and construct mapping dict

In [51]:
pos_tags = []
for i in data["Sentence"]:
  sen_tags = pos_tag(i.split())
  for word, j in sen_tags:
    if j not in pos_tags:
      pos_tags.append(j)
pos_tags_to_index = {pos_tags[i]:i for i in range(len(pos_tags))}
pos_tags_to_index

{'NNS': 0,
 'IN': 1,
 'VBP': 2,
 'VBN': 3,
 'NNP': 4,
 'TO': 5,
 'VB': 6,
 'DT': 7,
 'NN': 8,
 'CC': 9,
 'JJ': 10,
 '.': 11,
 'VBD': 12,
 'WP': 13,
 'PRP': 14,
 'CD': 15,
 'VBZ': 16,
 'POS': 17,
 'RB': 18,
 ',': 19,
 'VBG': 20,
 'WRB': 21,
 'PRP$': 22,
 'MD': 23,
 'WDT': 24,
 'NNPS': 25,
 'JJR': 26,
 ':': 27,
 'JJS': 28,
 'RP': 29,
 'WP$': 30,
 'PDT': 31,
 'EX': 32,
 'RBS': 33,
 '(': 34,
 ')': 35,
 '$': 36,
 'RBR': 37,
 'FW': 38,
 "''": 39,
 'SYM': 40,
 'UH': 41,
 '#': 42}

## function to map the original dataset with the tokenizer output

In [24]:
dataset = []
labels = []

with tqdm(total=len(data["Sentence"])) as pbar:
  for sentence, tag in zip(data["Sentence"],  data["Tag"]):
    pbar.set_description("Constructing Dataset")
    try:
      one_hot = []
      label = []
      tag = tag.replace("'","").replace('[','').replace(']','').replace(" ","").split(',')
      j=0
      embedding, tokens = get_embeddings([sentence.replace("-", "").replace("~", "")])

      for i, token in enumerate(tokens):

        if (token ==',' ) and tokens[i - 1].isdigit() and tokens[i + 1].isdigit():
          label.append('O')

        elif (token =='.' ) and tokens[i - 1].isdigit():
          label.append('O')
        elif  bool(re.search("^[a-z]+(-[a-z]+)+$", sentence.lower().split()[j])):
          label.append(tag[j])
          if token == sentence.lower().split()[j].split("-")[1]:
            j+=1
        elif  "'" in tokens[i-1] and bool(re.search("^[a-z]{1,3}$" , token)):
          label.append('O')
        elif  "'" in token and bool(re.search("^[a-z]{1,3}$" , tokens[i+1])):
          label.append(tag[j])

        elif  bool(re.search('([a-z]\.)+', sentence.lower().split()[j]))and token in sentence.lower().split()[j] :
          if tokens[i] == sentence.lower().split()[j][0]:
            label.append(tag[j])
          else:
            if tag[j] == "O":
              label.append(tag[j])
            else:
              label.append('I-'+tag[j].split('-')[1])

          if tokens[i-1] == sentence.lower().split()[j][-2]:
            j+=1
        elif token in sentence.lower().split():
          label.append(tag[j])
          j+=1

        else:
          if "##" not in token and not token.isdigit():
            label.append(tag[j])
            j+=1
          elif "##" in token:
            if tag[j-1] == "O":
              label.append(tag[j-1])
            else:
              label.append('I-'+tag[j-1].split('-')[1])
          else:
            label.append('O')
      sen_tags = pos_tag(tokens)
      for _, pos_tag_ in sen_tags:
        t = torch.zeros(len(pos_tags))
        t[pos_tags_to_index[pos_tag_]] = 1
        one_hot.append(t)
      one_hot = torch.stack(one_hot)
      if (embedding.squeeze(0).shape[0])!= len(one_hot):
        continue
      data_point = torch.cat((embedding.squeeze(0), one_hot), dim=1)
      label = [taps_to_index[i] for i in label]
      if len(label) != data_point.shape[0]:
        continue
      dataset.append(data_point)
      labels.append(label)
      pbar.update(1)
    except IndexError:
      continue

Constructing Dataset: 100%|█████████▉| 47811/47959 [2:05:39<00:23,  6.34it/s]


In [25]:
len(dataset), len(labels)

(47811, 47811)

##Labels to one hot encodding form

In [26]:
one_hot_labels = []
for label in labels:
  one_hot = []
  for tag in label:
    t = torch.zeros(len(tags))
    t[tag - 1] = 1
    one_hot.append(t)
  one_hot = torch.stack(one_hot)
  one_hot_labels.append(one_hot)

In [28]:
max = 0
for i in dataset:
  if i.shape[0] > max:
    max = i.shape[0]

After constructing the labels the dataset need high memory but i showed the concept of reconstructing the dataset labels to fit the BERT  tokenizer

As a result of low memory resources i will use another dataset which is tokenized and labeled

#WNUT 17: Emerging and Rare entity recognition

##Load Dataset

In [10]:
wnut = load_dataset("wnut_17")

The repository for wnut_17 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wnut_17.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

##List of labels

In [11]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

##Load tokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

##Example of tokenizer usage

In [13]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '@',
 'paul',
 '##walk',
 'it',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'i',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'empire',
 'state',
 'building',
 '=',
 'es',
 '##b',
 '.',
 'pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

## Align the labels with the text after tokenized

In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## Map Dataset to the new labels

In [15]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [16]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'I-location',
 'O',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

## Dataset into batches

In [17]:
def dataset_to_batches(data_, batch_size):
  dataset = []
  batch = []
  batch_labels = []
  i=0
  for data_point in data_:
    data = get_embeddings(torch.tensor(data_point['input_ids']).unsqueeze(0),torch.tensor(data_point['attention_mask']).unsqueeze(0))
    labels = []
    for label in data_point['labels']:
      one_hot = torch.zeros(14)
      if label == -100:
        one_hot[13] = 1
      else:
        one_hot[label] = 1
      labels.append(one_hot)
    labels = torch.stack(labels)
    if i % batch_size == 0 and i != 0:
      dataset.append((batch, batch_labels))
      batch = []
      batch_labels = []
      batch.append(data.squeeze(0))
      batch_labels.append(labels)
    else:
      batch.append(data.squeeze(0))
      batch_labels.append(labels)
    i+=1
  return dataset

In [18]:
wnut_train = dataset_to_batches(tokenized_wnut['train'], 16)
wnut_val = dataset_to_batches(tokenized_wnut['validation'], 16)

In [19]:
wnut_test = dataset_to_batches(tokenized_wnut['test'], 16)

In [20]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
    13: "sptag"
}

In [64]:
def convert_labels_to_strings(labels, label_map=id2label):
    return [[label_map[label] for label in labels]]

#LSTM

##LSTM Model Class

In [88]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
      super().__init__()
      self.hidden_size = hidden_size
      self.num_layers = num_layers
      self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True, batch_first=True)
      self.fc1 = torch.nn.Linear(hidden_size * 2, 1024)
      self.fc2 = torch.nn.Linear(1024, 512)
      self.fc3 = torch.nn.Linear(512, output_size)

    def forward(self, x):
      h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size ).to(x.device)
      c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
      out, _ = self.lstm(x, (h0, c0))
      out = torch.relu(self.fc1(out))
      out = torch.dropout(out, p=0.5, train=True)
      out = torch.relu(self.fc2(out))
      out = torch.dropout(out, p=0.5, train=True)
      out = self.fc3(out)
      return out

## Model Hyper-prammeters

In [89]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMModel(768, 1024, 5, 14)
model.to(device)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.95)
epochs = 10

##Training loop

In [90]:
for epoch in range(epochs):
  print(f'{datetime.datetime.now()} Epoch [{epoch + 1}/{epochs}]', end=" ")
  epoch_loss = 0
  count = 0
  with tqdm(total=len(wnut_train)) as pbar:
    for batch in wnut_train:
      inputs, labels  = batch
      optimizer.zero_grad()
      for input, label in zip(inputs, labels):
        output = model(input.unsqueeze(0).to(device))
        loss_value = loss(output.squeeze(0), label.to(device))
        epoch_loss += loss_value.item()
        count += 1
        loss_value.backward()
      optimizer.step()
      pbar.update(1)
      pbar.set_description(f'Training Progress | loss: {(epoch_loss / count):.4f}')
  scheduler.step()
  test_loss = 0
  test_count = 0
  with torch.no_grad():
    with tqdm(total=len(wnut_val)) as pbar2:
      precision = 0
      recall = 0
      for batch in wnut_val:
        inputs, labels = batch
        for input, label in zip(inputs, labels):
          output = model(input.unsqueeze(0).to(device))
          loss_value = loss(output.squeeze(0), label.to(device))
          test_loss += loss_value.item()
          test_count += 1
          y_true = torch.argmax(label, dim=1).cpu().numpy().tolist()
          y_pred = torch.argmax(output, dim=2).cpu().numpy().tolist()
          y_true = convert_labels_to_strings(y_true)
          y_pred = convert_labels_to_strings(y_pred[0])
          if len(y_true[0]) != len(y_pred[0]):
            continue
          precision += precision_score(y_true, y_pred, average='macro', zero_division=0)
          recall += recall_score(y_true, y_pred, average='macro', zero_division=0)
        pbar2.set_description(f'Testing Progress | loss: {(test_loss / test_count):.4f} Precision: {(precision/test_count):.2f} Recall: {(recall/test_count):.2f}')
        pbar2.update(1)

2024-07-21 00:12:13.004508 Epoch [1/10] 

Training Progress | loss: 0.8373: 100%|██████████| 212/212 [03:44<00:00,  1.06s/it]
Testing Progress | loss: 0.9903 Precision: 0.24 Recall: 0.35: 100%|██████████| 63/63 [00:17<00:00,  3.57it/s]



2024-07-21 00:16:15.098848 Epoch [2/10] 

Training Progress | loss: 0.7639: 100%|██████████| 212/212 [03:43<00:00,  1.05s/it]
Testing Progress | loss: 0.9481 Precision: 0.59 Recall: 0.48: 100%|██████████| 63/63 [00:17<00:00,  3.58it/s]



2024-07-21 00:20:16.259737 Epoch [3/10] 

...

Training Progress | loss: 0.1108: 100%|██████████| 212/212 [03:42<00:00,  1.05s/it]
Testing Progress | loss: 0.9957 Precision: 0.62 Recall: 0.68: 100%|██████████| 63/63 [00:17<00:00,  3.59it/s]



2024-07-21 00:44:25.126938 Epoch [9/10] 

Training Progress | loss: 0.1038: 100%|██████████| 212/212 [03:43<00:00,  1.05s/it]
Testing Progress | loss: 1.2297 Precision: 0.60 Recall: 0.66: 100%|██████████| 63/63 [00:17<00:00,  3.58it/s]



2024-07-21 00:48:26.137580 Epoch [10/10] 

Training Progress | loss: 0.1005: 100%|██████████| 212/212 [03:43<00:00,  1.05s/it]
Testing Progress | loss: 0.9753 Precision: 0.61 Recall: 0.67: 100%|██████████| 63/63 [00:17<00:00,  3.59it/s]

##Test model

In [91]:
with torch.no_grad():
  with tqdm(total=len(wnut_test)) as pbar2:
    precision = 0
    recall = 0
    for batch in wnut_test:
      inputs, labels = batch
      for input, label in zip(inputs, labels):
        output = model(input.unsqueeze(0).to(device))
        loss_value = loss(output.squeeze(0), label.to(device))
        test_loss += loss_value.item()
        test_count += 1
        y_true = torch.argmax(label, dim=1).cpu().numpy().tolist()
        y_pred = torch.argmax(output, dim=2).cpu().numpy().tolist()
        y_true = convert_labels_to_strings(y_true)
        y_pred = convert_labels_to_strings(y_pred[0])
        if len(y_true[0]) != len(y_pred[0]):
          continue
        precision += precision_score(y_true, y_pred, average='macro', zero_division=0)
        recall += recall_score(y_true, y_pred, average='macro', zero_division=0)
      pbar2.set_description(f'Testing Progress | loss: {(test_loss / test_count):.4f} Precision: {(precision/test_count):.2f} Recall: {(recall/test_count):.2f}')
      pbar2.update(1)

Testing Progress | loss: 0.9501 Precision: 0.30 Recall: 0.34: 100%|██████████| 80/80 [00:33<00:00,  2.40it/s]


#FineTune Pre-Trained BERT

In [10]:
seqeval = evaluate.load("seqeval")

In [11]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [12]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [9]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [13]:
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="/content/Untitled_Folder",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

TrainOutput(global_step=426, training_loss=0.21604803022644328, metrics={'train_runtime': 3205.3508, 'train_samples_per_second': 2.118, 'train_steps_per_second': 0.133, 'total_flos': 91781128898820.0, 'train_loss': 0.21604803022644328, 'epoch': 2.0})

In [19]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [23]:
from transformers import pipeline

classifier = pipeline("ner", model="/content/Untitled_Folder/checkpoint-426")
classifier(text)

[{'entity': 'B-group',
  'score': 0.28209433,
  'index': 1,
  'word': 'the',
  'start': 0,
  'end': 3},
 {'entity': 'B-location',
  'score': 0.50449383,
  'index': 2,
  'word': 'golden',
  'start': 4,
  'end': 10},
 {'entity': 'I-location',
  'score': 0.19977714,
  'index': 3,
  'word': 'state',
  'start': 11,
  'end': 16},
 {'entity': 'B-group',
  'score': 0.20830375,
  'index': 4,
  'word': 'warriors',
  'start': 17,
  'end': 25},
 {'entity': 'B-location',
  'score': 0.6363827,
  'index': 13,
  'word': 'san',
  'start': 80,
  'end': 83},
 {'entity': 'B-location',
  'score': 0.5648308,
  'index': 14,
  'word': 'francisco',
  'start': 84,
  'end': 93}]